In [1]:
!pip install -U azureml-sdk

Requirement already up-to-date: azureml-sdk in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (1.0.57)


You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.57


In [3]:
from azureml.core import Workspace
ws = Workspace(subscription_id="ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9", resource_group="DatasetTestRG", workspace_name="Dataset-Test-WS")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FUDNG5XE4 to authenticate.
Interactive authentication successfully completed.
Dataset-Test-WS
DatasetTestRG
eastus
ee85ed72-2b26-48f6-a0e8-cb5bcf98fbd9


In [4]:
from azureml.core.compute import AmlCompute
compute_target_name = "nlp-test"
compute_target = AmlCompute(ws, compute_target_name)

In [5]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.environment import DockerSection
runconfig = RunConfiguration(framework="python")
runconfig.environment.python.conda_dependencies = CondaDependencies('dependencies.yml')
runconfig.environment.docker = DockerSection()
runconfig.environment.docker.enabled = True
runconfig.environment.docker.gpu_support = True
runconfig.environment.docker.base_image = azureml.core.runconfig.DEFAULT_GPU_IMAGE

In [6]:
from azureml.data.data_reference import DataReference
from azureml.core import Datastore
ds = Datastore.get(ws, 'workspaceblobstore')

In [12]:
# # data module
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# # train_input_data = DataReference(
# #     datastore=ds,
# #     data_reference_name="train_input_data",
# #     path_on_datastore="train/cleaned_train_sampled.csv")
train_input_data = DataReference(
    datastore=ds,
    data_reference_name="train_input_data",
    path_on_datastore="debug/train_16.csv")
# train_data = PipelineData("train_data",datastore=ds)

# load_train_data_step = PythonScriptStep(
#     name="load_data_step",
#     script_name="run_modules/data_module.py", 
#     arguments=["--input_data", train_input_data, 
#                "--output_dir", train_data],
#     inputs=[train_input_data],
#     outputs=[train_data],
#     compute_target=compute_target,
#     runconfig = runconfig,
#     source_directory='.'
# )

In [13]:
# # data module —— load test dataset
# # test_input_data = DataReference(
# #     datastore=ds,
# #     data_reference_name="test_input_data",
# #     path_on_datastore="test/cleaned_test_data_labels.csv")
test_input_data = DataReference(
    datastore=ds,
    data_reference_name="test_input_data",
    path_on_datastore="debug/test_32.csv")

# test_data = PipelineData("test_data",datastore=ds)

# load_test_data_step = PythonScriptStep(
#     name="load_test_data_step",
#     script_name="run_modules/data_module.py", 
#     arguments=["--input_data", test_input_data, 
#                "--output_dir", test_data],
#     inputs=[test_input_data],
#     outputs=[test_data],
#     compute_target=compute_target,
#     runconfig = runconfig,
#     source_directory='.'
# )

In [14]:
#split data module
# dataset1 = PipelineData("output_data1",datastore=ds)
# dataset2 = PipelineData("output_data2",datastore=ds)
# #parameters
# row_split_rate = 0.7
# random = True
# random_seed = 200
# split_data_step = PythonScriptStep(
#     name="split_data_step",
#     script_name="run_modules/split_data_module.py", 
#     arguments=["--input_dir", train_data, 
#                "--output_dir1", dataset1,
#                "--output_dir2", dataset2,
#                "--row_split_rate", row_split_rate,
#                "--random", random,
#                "--random_seed", random_seed],
#     inputs=[train_data],
#     outputs=[dataset1, dataset2],
#     compute_target=compute_target,
#     runconfig = runconfig,
#     source_directory='.'
# )


In [15]:
# dataset1 = train_data
# dataset2 = test_data

In [16]:
# bert module
chosed_model = PipelineData("chosed_bert",datastore=ds)

language = "En"
uncased = True
build_bert_step = PythonScriptStep(
    name="build_bert_step",
    script_name="run_modules/bert_url_module.py", 
    arguments=["--out_model_dir", chosed_model,
               "--language", language,
              "--uncased", uncased],
    outputs=[chosed_model],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)


In [17]:
# custom added layer module
# label_num = 6
#category = 'lstm_multi_label_classifier' 
layer_name = 'multi_label_classifier'
added_layer = PipelineData("added_layer",datastore=ds)
custom_layer_step = PythonScriptStep(
    name="custom_layer_step",
    script_name="run_modules/add_layer_module.py", 
    arguments=["--layer_name", layer_name,
               "--output_dir", added_layer],
    outputs=[added_layer],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)


In [18]:
# train module 

trained_model_dir = PipelineData("trained_model_dir", datastore=ds)
is_training_bert= False # Device cannot support training and is generally setted False.
train_batch_size = 2
use_gpu = True
num_gpu_cores = 4
learning_rate = 5e-5
num_train_epochs = 3.0
train_column_names ="comment_text"
label_column_names = "toxic severe_toxic obscene threat insult identity_hate"
train_step = PythonScriptStep(
    name="train_step",
    script_name="run_modules/train_module.py", 
    arguments=["--bert_dir", chosed_model,
               "--added_layer_config", added_layer,
               "--train_data", train_input_data,
               "--output_dir", trained_model_dir,
               "--is_training_bert", is_training_bert,
               "--train_batch_size", train_batch_size,
               "--use_gpu",use_gpu,
               "--num_gpu_cores", num_gpu_cores,
               "--learning_rate", learning_rate,
               "--num_train_epochs", num_train_epochs,
               "--train_column_names",train_column_names,
               "--label_column_names", label_column_names],
    inputs=[chosed_model, added_layer, train_input_data],
    outputs=[trained_model_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)



In [19]:
# score/predict module

predict_result_dir = PipelineData("predict_dir",datastore=ds)
predict_batch_size = 2
use_gpu = True
num_gpu_cores = 4
predict_column_names = "comment_text"
predict_step = PythonScriptStep(
    name="predict_step",
    script_name="run_modules/predict_module.py", 
    arguments=["--test_data", test_input_data,
               "--trained_model_dir", trained_model_dir,
               "--output_dir", predict_result_dir,
               "--predict_batch_size", predict_batch_size,
               "--use_gpu", use_gpu,
               "--num_gpu_cores", num_gpu_cores,
               "--predict_column_names", predict_column_names],
    inputs=[test_input_data, trained_model_dir],
    outputs=[predict_result_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [20]:
# evaluate module
evaluate_result_dir = PipelineData("evaluate_results",datastore=ds)
pr = True
roc = True
label_columns = "toxic severe_toxic obscene threat insult identity_hate"
probability_columns = "probability_1 probability_2 probability_3 probability_4 probability_5 probability_6"
evaluate_step = PythonScriptStep(
    name="evaluate_step",
    script_name="run_modules/evaluate_module.py", 
    arguments=[ "--input_data", predict_result_dir,
                "--pr", pr,
                "--roc", roc,
                "--output_dir", evaluate_result_dir,
                "--label_columns", label_columns,
                "--probability_columns", probability_columns],
    inputs=[predict_result_dir],
    outputs=[evaluate_result_dir],
    compute_target=compute_target,
    runconfig = runconfig,
    source_directory='.'
)

In [21]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
pipeline_toxic= Pipeline(workspace=ws, steps=[evaluate_step])
#pipeline_split_data = Pipeline(workspace=ws, steps=[split_data_step])


In [22]:
pipeline_run = Experiment(ws, 'toxic_comment').submit(pipeline_toxic)
#pipeline_run = Experiment(ws, 'toxic_comment').submit(pipeline_split_data)

Created step evaluate_step [a1e2db07][98fec78d-7226-4350-9660-aba29abb8ecc], (This step will run and generate new outputs)
Created step predict_step [e1d6c2f2][2e258821-0159-4654-b5c8-1637a0db92c2], (This step will run and generate new outputs)
Created step train_step [737afc03][acf029aa-b4ca-4365-bdaf-50e8489e991b], (This step will run and generate new outputs)
Created step build_bert_step [8243b64a][bbafb92a-9a21-42a2-a0dc-34d87dca42c9], (This step will run and generate new outputs)
Created step custom_layer_step [fe687fd3][6783812a-e59c-4252-8a7f-2594a1cccc51], (This step will run and generate new outputs)
Using data reference test_input_data for StepId [87d28163][a191684b-1c2b-4289-9f01-1b69a7d71127], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_input_data for StepId [26b6056b][7975ef43-339a-49fe-a0cc-f23212c3204e], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: ae072fb3-fb0e-4c99-9430-275eb34c2c2c


In [21]:
from azureml.core import Experiment
from azureml.pipeline.core import PipelineRun
pipeline_run = Experiment(ws, 'toxic_comment')
pipeline_run = PipelineRun(pipeline_run, "0bf16d6b-1fc4-44a9-ab0f-f065256e01a8")
pipeline_run.cancel()
